In [ ]:
import scanpy as sc
import muon as mu
import numpy as np
import pandas as pd
import snapatac2 as snap
import os

In [ ]:
output_fol = '/home/carsten/alvira_bioinformatics/postnatal_lung_multiome/data/pilot/230525_snapatac2_pilot'
os.makedirs(output_fol, exist_ok=True)

In [ ]:
adata_rna = sc.read('/home/carsten/alvira_bioinformatics/postnatal_lung_multiome/data/single_cell_files/share/p7_multiome_rna.gz.h5ad')
adata_rna.obs_names

In [ ]:
atac_adata = snap.read('/home/carsten/alvira_bioinformatics/postnatal_lung_multiome/data/single_cell_files/share/p7_multiome_atac.gz.h5ad')

In [ ]:
atac_adata

In [ ]:
fragment_file = '/home/carsten/alvira_bioinformatics/postnatal_lung_multiome/data/cellranger_output/230609_aggregate/outs/atac_fragments.tsv.gz'
output_f = '/home/carsten/alvira_bioinformatics/postnatal_lung_multiome/data/single_cell_files/multiome_snapatac_pilot.h5ad'
gtf = '/home/carsten/alvira_bioinformatics/postnatal_lung_multiome/data/refdata-cellranger-arc-mm10-2020-A-2.0.0/genes/genes.gtf'
chrom_sizes_fn = '/home/carsten/alvira_bioinformatics/postnatal_lung_multiome/data/refdata-cellranger-arc-mm10-2020-A-2.0.0/sizes.genome'
genome = '/home/carsten/alvira_bioinformatics/postnatal_lung_multiome/data/refdata-cellranger-arc-mm10-2020-A-2.0.0/fasta/genome.fa'

In [ ]:
data = snap.read(output_f)
snap.pl.umap(data, color='celltype', interactive=False, height=500)


In [ ]:
data.obsm['X_umap']

In [ ]:
adata_rna = adata_rna[data.obs_names]
adata_rna

In [ ]:
for lineage in adata_rna.obs['lineage'].cat.categories:
    lin_data = data.subset(obs_indices=adata_rna[adata_rna.obs['lineage']==lineage].obs_names,
               out = f'{output_fol}/atac_{lineage}.h5ad')
    snap.pp.select_features(lin_data, n_features=10000)
    snap.pp.harmony(lin_data, batch='mouse')
    snap.tl.umap(lin_data, use_rep='X_spectral_harmony')
    snap.pp.knn(lin_data,use_rep ='X_spectral_harmony' )
    snap.tl.leiden(lin_data)
    snap.pl.umap(lin_data, color='celltype', interactive=False, height=500)
    snap.pl.umap(lin_data, color='treatment', interactive=False, height=500)
    snap.pl.umap(lin_data, color='mouse', interactive=False, height=500)


    lin_data.close()
    
    
    
    
    
    

In [ ]:
end_data = snap.read(f'{output_fol}/atac_endothelial.h5ad')
snap.pl.umap(end_data, color='celltype', interactive=True, height=500)


In [ ]:
snap.pl.umap(end_data, color='celltype', interactive=False, height=500)


In [ ]:
snap.pl.umap(end_data, color='treatment', interactive=False, height=500)


In [ ]:
snap.pl.umap(end_data, color='mouse', interactive=False, height=500)

In [ ]:
snap.pl.umap(end_data, color='leiden', interactive=False, height=500)

In [ ]:
mes_data = snap.read(f'{output_fol}/atac_mesenchymal.h5ad')
snap.pl.umap(mes_data, color='celltype', interactive=False, height=500)

In [ ]:
imm_data = snap.read(f'{output_fol}/atac_immune.h5ad')
snap.pl.umap(imm_data, color='celltype', interactive=False, height=500)

In [ ]:
epi_data = snap.read(f'{output_fol}/atac_epithelial.h5ad')
snap.pl.umap(epi_data, color='celltype', interactive=False, height=500)

In [ ]:
epi_data.var['selected'].to_pandas().value_counts()

In [ ]:
snap.pl.umap(data, color='mouse', interactive=False, height=500)


In [ ]:
diff_peaks_dt = {}
for ct in adata_rna.obs['celltype'].cat.categories:
    print(ct)
    norm = (data.obs['celltype'] == ct)&(data.obs['treatment'] == 'Normoxia')
    hyper = (data.obs['celltype'] == ct)&(data.obs['treatment'] == 'Hyperoxia')
    peaks_selected = np.logical_or(
    data.uns["peaks"][ct].to_numpy(),
    data.uns["peaks"][ct].to_numpy(),
)

    diff_peaks = snap.tl.diff_test(
    peak_mat,
    cell_group1=norm,
    cell_group2=hyper,
        features=data.uns["peaks"][ct].to_numpy()
)
    diff_peaks = diff_peaks.to_pandas()
    difF_peaks_dt[ct] = diff_peaks

In [ ]:
norm = (data.obs['celltype'] == 'AT1')&(data.obs['treatment'] == 'Normoxia')
hyper = (data.obs['celltype'] == 'AT2')&(data.obs['treatment'] == 'Normoxia')
peaks_selected = np.logical_or(
data.uns["peaks"]['AT1'].to_numpy(),
data.uns["peaks"]['AT2'].to_numpy(),
)

diff_peaks = snap.tl.diff_test(
peak_mat,
cell_group1=norm,
cell_group2=hyper,
    features=peaks_selected,
    penalty='none'
)
diff_peaks = diff_peaks.to_pandas()
difF_peaks_dt

In [ ]:
snap.pl.motif_enrichment(motifs, max_fdr=0.0001, height=3000, width = 2000,interactive=False)


In [ ]:
rna_names=adata_rna.obs_names.tolist()
overlap_names = list(set(adata_rna.obs_names) & set(data.obs_names))
not_in_fragment = [x for x in rna_names if x not in overlap_names]

In [ ]:
adata_rna.obs.loc[not_in_fragment]

In [ ]:
not_in_fragment

In [ ]:
print(len(adata_rna.obs_names.tolist()))
print(len(data.obs_names))
print(len(list(set(adata_rna.obs_names) & set(data.obs_names))))


In [ ]:
snap.pl.tsse(data, interactive=False)

In [ ]:
data

In [ ]:
len(set(data.obs_names))

In [ ]:
data_sub = 
data_sub

In [ ]:
%%time
snap.pp.filter_cells(data, min_counts=5000, min_tsse=0, max_counts=1000000)
data